# Molecular Simulation 
Carrying out molecular dynamics (MD) and Monte-Carlo (MC) simulations using a trained potential. 

In [1]:
from utils import set_env
# set_env('.env')

In [2]:
import os
os.environ["JAX_ENABLE_X64"] = "1"
os.environ["JAX_PLATFORM_NAME"] = "cpu" 
os.environ["XLA_PYTHON_CLIENT_PREALLOCATE"] = "false" 

In [3]:
from jaxip.datasets import RunnerDataset
from jaxip.potentials import NeuralNetworkPotential
from jaxip.simulation import (
    MDSimulator, 
    BrendsenThermostat, 
    MCSimulator,
    run_simulation
)
from jaxip.atoms import Structure
from jaxip.units import units as units

import matplotlib.pylab as plt
from pathlib import Path
from ase import Atoms
from ase.visualize import view
import ase.io
import jax.numpy as jnp

In [4]:
base_dir = Path('LJ')

## Data

In [5]:
d = 6  # Angstrom
uc = Atoms('He', positions=[(d/2, d/2, d/2)], cell=(d, d, d))
s0 = Structure.from_ase(uc.repeat((7, 7, 7)))

# d = 10  # Angstrom
# uc = Atoms('Ar', positions=[(d/2, d/2, d/2)], cell=(d, d, d))
# s0 = Structure.create_from_ase(uc.repeat((7, 7, 7)))

atoms = s0.to_ase()
# view(atoms, viewer='x3d') # ase, ngl

2023-07-02 12:42:06.194170: E external/xla/xla/stream_executor/cuda/cuda_driver.cc:268] failed call to cuInit: CUDA_ERROR_UNKNOWN: unknown error


## Potential

In [6]:
from jaxip.types import Array
import jax
from functools import partial

@partial(jax.jit, static_argnums=0)
def _compute_pair_energy(obj, r: Array) -> Array:
    term = obj.sigma / r
    term6 = term**6
    return 4.0 * obj.epsilon * term6 * (term6 - 1.0)


@partial(jax.jit, static_argnums=0)
def _compute_pair_force(obj, r: Array, R: Array) -> Array:
    term = obj.sigma / r
    term6 = term**6
    force_factor = -24.0 * obj.epsilon / (r * r) * term6 * (2.0 * term6 - 1.0)
    return jnp.expand_dims(force_factor, axis=-1) * R


class LJPotential:
    def __init__(
        self,
        sigma: float,
        epsilon: float,
        r_cutoff: float,
    ) -> None:
        self.sigma = sigma
        self.epsilon = epsilon
        self.r_cutoff = r_cutoff

    def __call__(self, structure: Structure) -> Array:
        r, _ = structure.calculate_distance(atom_indices=jnp.arange(structure.natoms))
        mask = (0 < r) & (r < self.r_cutoff)
        pair_energies = _compute_pair_energy(self, r)
        return 0.5 * jnp.where(mask, pair_energies, 0.0).sum()

    def compute_forces(self, structure: Structure) -> Array:
        r, R = structure.calculate_distance(atom_indices=jnp.arange(structure.natoms))
        mask = (0 < r) & (r < self.r_cutoff)
        pair_forces = jnp.where(
            jnp.expand_dims(mask, axis=-1),
            _compute_pair_force(self, r, R),
            jnp.zeros_like(R),
        )
        return jnp.sum(pair_forces, axis=1)


# He
ljpot = LJPotential(
    sigma=2.5238 * units.FROM_ANGSTROM,  # Bohr
    epsilon=4.7093e-04 * units.FROM_ELECTRON_VOLT,  # Hartree
    r_cutoff=6.3095 * units.FROM_ANGSTROM,  # 2.5 * sigma
)

# Ar
# ljpot = LJPotential(
#     sigma=3.405 * units.FROM_ANGSTROM,                       # Bohr
#     epsilon=0.01032439284 * units.FROM_ELECTRON_VOLT,        # Hartree
#     r_cutoff=8.5125 * units.FROM_ANGSTROM,                   # 2.5 * sigma
# )

## Molecular Dynamics (MD)

In [7]:
# v0 = MDSimulator.generate_random_velocity(temperature=300.0, mass=s0.mass, seed=2023)
# brendsen = BrendsenThermostat(target_temperature=300.0, time_constant=50.0 * units.FROM_FEMTO_SECOND)

md = MDSimulator(
    potential=ljpot,
    initial_structure=s0,
    time_step=0.5 * units.FROM_FEMTO_SECOND,
    temperature=300, # K
    # initial_velocity=v0,
    # thermostat=brendsen
)

In [8]:
md.get_pressure()

Array(6.51659916e-07, dtype=float64)

In [9]:
md.get_total_energy()

Array(0.48832374, dtype=float64)

In [10]:
# Warmp up
# run_simulation(md)

# %timeit md.run_simulation(num_steps=1, output_freq=-1)

In [11]:
run_simulation(md, num_steps=10000, output_freq=100, filename="md.xyz")

0          time[ps]:0.00000    Temp[K]:299.95012  Etot[Ha]:0.4883237353    Epot[Ha]:-0.0003923654   Pres[kb]:0.19173   
100        time[ps]:0.05000    Temp[K]:180.48925  Etot[Ha]:0.2937277968    Epot[Ha]:-0.0003477770   Pres[kb]:0.11535   
200        time[ps]:0.10000    Temp[K]:340.79198  Etot[Ha]:0.5549463578    Epot[Ha]:-0.0003143736   Pres[kb]:0.21783   
300        time[ps]:0.15000    Temp[K]:332.48107  Etot[Ha]:0.5414355956    Epot[Ha]:-0.0002839803   Pres[kb]:0.21252   
400        time[ps]:0.20000    Temp[K]:339.34123  Etot[Ha]:0.5526201298    Epot[Ha]:-0.0002768727   Pres[kb]:0.21688   
500        time[ps]:0.25000    Temp[K]:325.52335  Etot[Ha]:0.5301128543    Epot[Ha]:-0.0002703262   Pres[kb]:0.20803   
600        time[ps]:0.30000    Temp[K]:323.53786  Etot[Ha]:0.5268878850    Epot[Ha]:-0.0002602887   Pres[kb]:0.20678   
700        time[ps]:0.35000    Temp[K]:298.66455  Etot[Ha]:0.4863645128    Epot[Ha]:-0.0002569646   Pres[kb]:0.19088   
800        time[ps]:0.40000    Temp[K]:3

In [12]:
md.get_pressure() * units.TO_KILO_BAR

Array(0.20357661, dtype=float64)

In [13]:
md.positions * units.TO_ANGSTROM

Array([[ 4.64015904,  3.90580761,  8.96299329],
       [40.43744741, 41.79151308, 13.22102852],
       [ 0.94542291,  8.79409743, 12.07380893],
       ...,
       [33.22353222, 38.70496147, 23.33706654],
       [ 0.73022476, 36.42934922, 31.8933132 ],
       [33.66265392, 30.98371204, 32.19107278]], dtype=float64)

In [14]:
atoms = md.get_structure().to_ase()
# view(atoms, viewer='x3d') # ase, ngl

## Monte Carlo (MC)

In [15]:
mc = MCSimulator(
    potential=ljpot,
    initial_structure=s0,
    temperature=300, # K
    translate_step=0.3 * units.FROM_ANGSTROM,
    movements_per_step=10,
)

In [16]:
# Warmp up
run_simulation(mc)

# %timeit run_simulation(mc, num_steps=1, output_freq=-1)

0          Temp[K]:300.0000000000  Epot[Ha]:-0.0003923654   


/home/hossein/miniconda3/envs/jaxip/lib/python3.10/site-packages/jax/_src/ops/scatter.py:89: FutureWarning: scatter inputs have incompatible types: cannot safely cast value from dtype=float64 to dtype=float32. In future JAX releases this will result in an error.
  warnings.warn("scatter inputs have incompatible types: cannot safely cast "


1          Temp[K]:300.0000000000  Epot[Ha]:-0.0003924740   


In [17]:
run_simulation(mc, num_steps=10000, output_freq=100, filename=Path("mc.xyz"))

1          Temp[K]:300.0000000000  Epot[Ha]:-0.0003924740   
101        Temp[K]:300.0000000000  Epot[Ha]:-0.0003682883   
201        Temp[K]:300.0000000000  Epot[Ha]:-0.0003823111   
301        Temp[K]:300.0000000000  Epot[Ha]:-0.0004225352   
401        Temp[K]:300.0000000000  Epot[Ha]:-0.0004656863   
501        Temp[K]:300.0000000000  Epot[Ha]:-0.0005311535   
601        Temp[K]:300.0000000000  Epot[Ha]:-0.0005888058   
701        Temp[K]:300.0000000000  Epot[Ha]:-0.0006248421   
801        Temp[K]:300.0000000000  Epot[Ha]:-0.0006788228   
901        Temp[K]:300.0000000000  Epot[Ha]:-0.0007861324   
1001       Temp[K]:300.0000000000  Epot[Ha]:-0.0008446493   
1101       Temp[K]:300.0000000000  Epot[Ha]:-0.0005072964   
1201       Temp[K]:300.0000000000  Epot[Ha]:-0.0006377003   
1301       Temp[K]:300.0000000000  Epot[Ha]:-0.0008396047   
1401       Temp[K]:300.0000000000  Epot[Ha]:0.0001787138    
1501       Temp[K]:300.0000000000  Epot[Ha]:0.0002712560    
1601       Temp[K]:300.0

In [ ]:
mc.positions * units.TO_ANGSTROM

In [20]:
atoms = mc.get_structure().to_ase()
# view(atoms, viewer='x3d') # ase, ngl